In [56]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import yaml

from typing import Dict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

%load_ext kedro.ipython
device = torch.device('mps')

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


Write functions to check if each node produces valid data that are suitable for next step

# Functions to combine dataset

In [35]:
# get data between exp_no 104 to 113
# append data of 105 at the end of 104 etc

def get_data(exp_no) -> pd.DataFrame:
    file_name = f"{exp_no}_SHT_SMD.txt"
    file_path = f"../data/01_raw/{file_name}"
    df = pd.read_csv(file_path, sep=',', usecols=['timestamp', 'SHT40_temp', 'SHT40_Humidity', 'A1_Sensor', 'A1_Resistance'])
    return df

def concat_data(start:int,end:int) -> pd.DataFrame:
    df = pd.concat([get_data(exp_no) for exp_no in range(start, end)])
    df = df.reset_index(drop=True)
    return df

s_file = 108
e_file = 113

concat_data = concat_data(s_file,e_file)
# concat_data(s_file,e_file).to_parquet(f'../data/02_intermediate/{s_file}_{e_file}.pq')

# Data processing node

In [36]:
def _hi_lo_peak(x: pd.DataFrame) -> pd.DataFrame:
    peaks, properties = find_peaks(x['A1_Sensor'], width=50, height=1)
    peak_heights = properties['peak_heights']
# Determine smaller and larger peaks
    smaller_peaks, larger_peaks = [], []
    for i in range(len(peaks) - 1):
        if peak_heights[i] > peak_heights[i + 1]:
            larger_peaks.append(peaks[i])
            smaller_peaks.append(peaks[i + 1])
    # smaller_peaks_df = x.iloc[smaller_peaks]
    return smaller_peaks

def data_stack(sp: pd.DataFrame, df: pd.DataFrame) -> pd.DataFrame:
    """
    After finding the peaks, stack the data according to exp_no
    """
    df_stacked_list = []
    for i in range(len(sp) - 1):
        df_subset = df.iloc[sp[i]:sp[i + 1]].copy()
        df_subset['exp_no'] = i
        df_subset['timestamp'] -= df_subset['timestamp'].iloc[0]
        df_stacked_list.append(df_subset)
        df_stacked = pd.concat(df_stacked_list, ignore_index=True)
    return df_stacked


def _group_by_bin(df_stacked: pd.DataFrame, num_bins: int) -> pd.DataFrame:
    """
    Use PD.CUT to group data into specified bins in parameters
    """
    df_list = []
    grouped = df_stacked.groupby('exp_no')
    for name, group in grouped:
        group['bin'] = pd.cut(group['timestamp'], bins=num_bins, labels=False)
        df_list.append(group)
    return pd.concat(df_list)

def _average_bin(bin_df: pd.DataFrame) -> pd.DataFrame:
    """
    average values within each bin to return only one data point
    """
    bin_df = bin_df.drop(columns=['timestamp'])
    grouped = bin_df.groupby(['exp_no', 'bin']).mean()
    return grouped.reset_index()

def preprocess_data_bin(mox: pd.DataFrame, num_bins: int) -> pd.DataFrame:
    """
    Return data that is sorted by experiment number according to lo_peak interval
    data is stacked and labeled by exp_no
    data is grouped by bin and averaged
    """
    df_stacked = data_stack(_hi_lo_peak(mox), mox)
    bin_df = _group_by_bin(df_stacked, num_bins)
    mean_bin = _average_bin(bin_df)
    return mean_bin

def get_percentile_data(df, percentile):
    """
    Returns the data up to the specified percentile based on the 'bin' column.

    :param df: DataFrame containing the data
    :param percentile: A float value between 0 and 1 representing the percentile
    :return: DataFrame containing the data up to the specified percentile
    """
    # Calculate the bin index corresponding to the percentile
    max_bin = int(percentile * df['bin'].max())

    # Return data up to that bin
    return df[df['bin'] <= max_bin]

def _group_percentile (averaged: pd.DataFrame, percentile_bins: float) -> pd.DataFrame:
    """
    Returns the full specified percentile dataset
    """
    df_list = []
    grouped = averaged.groupby('exp_no')
    for name, group in grouped:
        percentile_data = get_percentile_data(group, percentile_bins)
        df_list.append(percentile_data)
    return pd.concat(df_list)

def _transpose_(df_set: pd.DataFrame) -> pd.DataFrame:
    transposed = df_set.pivot(index='exp_no', columns='bin', values='A1_Resistance')
    transposed.columns = ['bin_' + str(col) for col in transposed.columns]
    transposed.reset_index(inplace=True)
    return transposed


def _res_ratio(averaged: pd.DataFrame) -> pd.DataFrame:
    def calculate_res_ratio(group):
        return group['A1_Resistance'].max() / group['A1_Resistance'].min()

    res_ratio = averaged.groupby('exp_no').apply(calculate_res_ratio).reset_index()
    res_ratio.columns = ['exp_no', 'res_ratio']
    return res_ratio

def _combine_feature_matrix(res_ratio: pd.DataFrame, transposed: pd.DataFrame) -> pd.DataFrame:
    combined = pd.merge(res_ratio, transposed, on='exp_no')
    return combined

def create_model_input_table(mox_bin: pd.DataFrame, percentile_bins: float) -> pd.DataFrame:
    selected_range = _group_percentile(mox_bin, percentile_bins)
    # the ratio is from the entire dataset not filtered to be ground truth
    res_ratio = _res_ratio(mox_bin) 
    transpose_col = _transpose_(selected_range)
    # drop exp_no to avoid training on exp_no
    mox_table = _combine_feature_matrix(transpose_col, res_ratio).drop(columns=['exp_no'])
    return mox_table

# Parameters

In [98]:
with open('nb_parameters.yml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

test_size = parameters['model_options']['test_size']

print(test_size)


# Hyper-parameters 

num_classes = parameters['model_options']['num_classes']
num_epochs = parameters['model_options']['num_epochs']
batch_size = parameters['model_options']['batch_size']
learning_rate = parameters['model_options']['learning_rate']

"""

Each feature as a time step in your sequence, you could set sequence_length to 150 and input_size to 1.
This would mean you are feeding in sequences of length 150, with each time step in the sequence having 1 feature.

"""

input_size = int(parameters['model_options']['input_size'])
# sequence_length = parameters['model_options']['sequence_length'] # the window it trains with can be selected
hidden_size = parameters['model_options']['hidden_size']
num_layers = parameters['model_options']['num_layers']
random_state = parameters['model_options']['random_state']

0.2


---
# Process and examine each file

In [99]:
exp_no = 107
percentile_bins = parameters['percentile_bins']
bin_size = int(parameters['num_bins'])
sequence_length = int(percentile_bins*bin_size)

df_exp = get_data(exp_no)
smaller_peaks = _hi_lo_peak(df_exp)
df_stacked = data_stack(smaller_peaks, df_exp)
bin_df = _group_by_bin(df_stacked, bin_size)
mean_bin = _average_bin(bin_df)
mox_bin = preprocess_data_bin(df_exp, bin_size)
selected_range = _group_percentile(mox_bin, percentile_bins)
res_ratio = _res_ratio(mox_bin)
transpose_col = _transpose_(selected_range)
mox_table = _combine_feature_matrix(transpose_col, res_ratio).drop(columns=['exp_no'])


---
# LSTM Code

In [100]:
# Implement LSTM functions below
# there is no validation set in this example
# load mox_table as input

def split_data(model_input_table: pd.DataFrame) -> torch.tensor:
    # print(f"Test size: {parameters['test_size']}, type: {type(parameters['test_size'])}")
    # print(f"Random state: {parameters['random_state']}, type: {type(parameters['random_state'])}")

    # Split data into features and target
    X = model_input_table[model_input_table.columns[:-1]].values  # Assuming last column is the target
    y = model_input_table[model_input_table.columns[-1]].values
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# check for NaN values
    # Convert numpy arrays back to DataFrame for easier manipulation
    X_train_df = pd.DataFrame(X_train, columns=model_input_table.columns[:-1])
    # Fill NaN values with the mean of the column
    X_train_df.fillna(X_train_df.mean(), inplace=True)
# Convert back to numpy arrays
    X_train = X_train_df.values

# Check for nan or infinite values
    # print("Number of NaN values in training data:")
    # print(X_train_df.isna().sum())
    # print("\nNumber of infinite values in training data:")
    # print(np.isinf(X_train_df).sum())

    # Initialize StandardScaler
    scaler = StandardScaler()
    # Fit on training data
    scaler.fit(X_train)
    # Transform both training and testing data
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print("Mean of scaled training features:")
    print(X_train_scaled.mean())
    print("\nStandard deviation of scaled training features:")
    print(X_train_scaled.std())

    print("\nMean of scaled testing features:")
    print(X_test_scaled.mean())
    print("\nStandard deviation of scaled testing features:")
    print(X_test_scaled.std())

    # Ensure y_train and y_test are in the correct format
    if isinstance(y_train, pd.Series):
        y_train = y_train.values
    if isinstance(y_test, pd.Series):
        y_test = y_test.values
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32))
    y_train_tensor = torch.tensor(y_train.astype(np.float32))
    X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32))
    y_test_tensor = torch.tensor(y_test.astype(np.float32))

    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

# create X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor from split_data(df)
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = split_data(mox_table)
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# Initialize DataLoaders
batch_size = parameters['model_options']['batch_size']  # You can adjust the batch size according to your needs
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Mean of scaled training features:
1.7490282726402467e-17

Standard deviation of scaled training features:
1.0

Mean of scaled testing features:
0.045656781875239985

Standard deviation of scaled testing features:
1.0021147944251179


In [101]:
# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        # out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        # out: (n, 10)
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# turn the block below into a function
def train_model (data: DataLoader)->():
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
    n_total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for i, (bins, target) in enumerate(train_loader):  
            bins = bins.reshape(-1, sequence_length, input_size).to(device)
            target = target.to(device)
        
        # Forward pass
        outputs = model(bins)
        # Example of reshaping/squeezing if applicable
        outputs = outputs.squeeze()  # Removes dimensions of size 1
        outputs = outputs[:64]  # Adjust if you need to slice the outputs

        target = target.unsqueeze(1).to(device)  # Add an extra dimension to match outputs
        loss = criterion(outputs, target)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

    # Calculate RMSE at the end of each epoch
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Don't calculate gradients
            total_loss = 0
            count = 0
            for bins, target in test_loader:  # Replace with your validation loader
                bins = bins.reshape(-1, sequence_length, input_size).to(device)
                target = target.unsqueeze(1).to(device)  # Add an extra dimension to match outputs
                outputs = model(bins)
                loss = criterion(outputs, target)
                total_loss += loss.item()
                count += 1
            rmse = np.sqrt(total_loss / count)
            print(f'Epoch [{epoch+1}/{num_epochs}], RMSE on validation data: {rmse}')
        model.train()  # Set the model back to training mode
    # Save the model after training
    # lstm_model = torch.save(model.state_dict())
    lstm_model = model.state_dict()
    return lstm_model

In [102]:
train_model(train_loader)

[01/02/24 16:56:56] WARNING  /Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site- ]8;id=515416;file:///Users/kpt/.pyenv/versions/3.10.13/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=65363;file:///Users/kpt/.pyenv/versions/3.10.13/lib/python3.10/warnings.py#109\109]8;;\
                             packages/torch/nn/modules/loss.py:535: UserWarning: Using a target                    
                             size (torch.Size([3, 1])) that is different to the input size                         
                             (torch.Size([3])). This will likely lead to incorrect results due to                  
                             broadcasting. Please ensure they have the same size.                                  
                               return F.mse_loss(input, target, reduction=self.reduction)                          
                                                                                                                   

Epoch [1/200], RMSE on validation data: 1.3883113473561048
Epoch [2/200], RMSE on validation data: 1.3845442795428287
Epoch [3/200], RMSE on validation data: 1.3807977773262823
Epoch [4/200], RMSE on validation data: 1.3770303258153682
Epoch [5/200], RMSE on validation data: 1.3732450838672887
Epoch [6/200], RMSE on validation data: 1.369461849040575
Epoch [7/200], RMSE on validation data: 1.365734919962479
Epoch [8/200], RMSE on validation data: 1.3619899372479718
Epoch [9/200], RMSE on validation data: 1.3582407176246314
Epoch [10/200], RMSE on validation data: 1.3544710648587333
Epoch [11/200], RMSE on validation data: 1.350708520805582
Epoch [12/200], RMSE on validation data: 1.3469031842108508
Epoch [13/200], RMSE on validation data: 1.3430509301340132
Epoch [14/200], RMSE on validation data: 1.3391873834479704
Epoch [15/200], RMSE on validation data: 1.3352750507712476
Epoch [16/200], RMSE on validation data: 1.3313104127778275
Epoch [17/200], RMSE on validation data: 1.327301836


OrderedDict([('lstm.weight_ih_l0', tensor([[ 6.5383e-02],
        [-3.6364e-02],
        [-2.5564e-03],
        [-4.1039e-02],
        [-5.4761e-02],
        [ 1.1671e-02],
        [-4.5836e-02],
        [ 5.2620e-02],
        [-7.3188e-02],
        [ 7.8954e-02],
        [-5.8750e-02],
        [ 3.9181e-02],
        [-2.5119e-02],
        [ 8.6830e-03],
        [ 5.5905e-02],
        [-6.1097e-02],
        [-3.6719e-02],
        [ 4.2161e-02],
        [-9.1778e-02],
        [ 4.8397e-02],
        [ 1.3639e-02],
        [-7.4973e-02],
        [ 5.6265e-02],
        [ 5.4310e-02],
        [ 5.7243e-02],
        [-5.5832e-02],
        [-4.1797e-02],
        [-7.5882e-02],
        [-3.1519e-03],
        [-4.1398e-02],
        [ 6.7614e-02],
        [-5.8483e-02],
        [ 4.8993e-03],
        [-8.3556e-02],
        [ 8.1682e-02],
        [-4.8920e-02],
        [ 5.0515e-03],
        [ 6.6139e-02],
        [-7.9427e-02],
        [-4.7259e-02],
        [ 7.1595e-02],
        [-5.1858e-02]